In [1]:

import pyodbc
import plotly
import plotly_express as px
import plotly.graph_objects as go
import pandas as pd
import json


server = 'D6SMTCV2'

database= 'DB004'

username = 'favalos'
password = 'favalos'
connection = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')



In [2]:
cursor = connection.cursor()
SQL = ( "SELECT j.Attribute,j.Desc1,COUNT(j.Attribute) as count "
            "FROM dbo.Journal as j "
            "WHERE j.Date_Time >= ?  and j.Date_Time <=  ?  and j.Module = ? "
            "GROUP BY j.Attribute, j.Desc1 "
            "ORDER BY count DESC")
fecha_i = '2022-01-01 00:00:00'
fecha_f = '2022-01-31 23:59:59'
module = '346PIC528'
cursor.execute(SQL,(fecha_i,fecha_f,module))
datos = cursor.fetchall()
cursor.close()

In [7]:
#pd.read_csv('datos_attr_desc1_count.csv')

df = pd.DataFrame()

df['Attribute'] = [r[0] for r in datos]
df['Desc1']     = [r[1] for r in datos]
df['count']     = [r[2] for r in datos]

#Set name of columns
df.columns = ['Attribute','Desc1','count']

#Find the uniques
uniques_attributes = df['Attribute'].unique()

list_df = []

for attr in uniques_attributes:
    df_aux = df[df['Attribute'].str.contains(attr)]
    list_df.append(df_aux)



<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [13]:
for i in range(len(list_df)):
    df_aux = list_df[i]
    for index,rows in df_aux.iteritems():
        print(rows['Attribute'], rows['Desc1'], rows['count'])
    
        

KeyError: 'Attribute'

Agrupar modulo para conocer la frecuencia de los datos

In [129]:
#Agrupar por modulo

freqModules = df.groupby(['module']).size()
freqModules = freqModules.sort_values(ascending=False)
freqModules = freqModules.index[:10]        #Obtener los 10 modulos con mas cambios


 
#Filtrar por los 10 modulos con mas cambios en un nuevo dataframe
df2 = df[df['module'].isin(freqModules)]
df2 = df2.groupby(['module','desc1']).size()
#Crea un dataframe con los 10 modulos con mas cambios
df2 = df2.to_frame()
df2 = df2.reset_index()
df2.columns = ['module','desc1','count']


#Grafica de barras
fig = px.bar(df2, x="module", y="count", color="desc1", title="Modulos con mas cambios", labels={"module":"Modulo","count":"Numero de cambios","desc1":"Usuario"})
fig.show()


